## 전치행렬과 max 값을 이용하여 데이터 Normalize 한 후,
## training data 와 test data 분리

In [1]:
import numpy as np
from datetime import datetime

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [2]:
# ThoracicSurgery Class

class ThoracicSurgery:
    
    # 생성자
    # xdata, tdata => numpy.array(...)
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        
        # 2층 hidden layer unit 
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(input_nodes, hidden_nodes)  
        self.b2 = np.random.rand(hidden_nodes)
        
        # 3층 output layer unit : 1 개 
        self.W3 = np.random.rand(hidden_nodes,output_nodes)
        self.b3 = np.random.rand(output_nodes)
                        
        # 학습률 learning rate 초기화
        self.learning_rate = learning_rate
        
        print("ThoracicSurgery object is created !!!")
        
    # 손실함수
    def feed_forward(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # obtain W and b
    def get_W_b(self):
        
        return self.W2,  self.b2, self.W3, self.b3
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # query, 즉 미래 값 예측 함수
    def predict(self, input_data):    
        
        z2 = np.dot(input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

    
    def accuracy(self, input_data, target_data):
        
        matched_list = []
        not_matched_list = []
        
        # list which contains (index, label, prediction) value
        index_label_prediction_list = []
        
        # temp list which contains label and prediction in sequence
        temp_list = []
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
                temp_list.append(index)
                temp_list.append(target_data[index])
                temp_list.append(logical_val)
                
                index_label_prediction_list.append(temp_list)
                
                temp_list = []
                
                
        accuracy_result = len(matched_list) / len(input_data)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list, index_label_prediction_list
    
        
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self, input_data, target_data):
        
        self.input_data = input_data
        self.target_data = target_data
        
        f = lambda x : self.feed_forward()
        
        self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
    
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        
        self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
    
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)

## ThoracicSurgery.csv 를 읽어 온 후에,
## 각 열의 최대값을 구하기 위해 원본 행렬을 전치행렬로 변환함

In [3]:
# Normalize 않되어 있는 ThoracicSurgery.csv 읽어옴
loaded_data = np.loadtxt('./ThoracicSurgery.csv', delimiter=',', dtype=np.float32)

print("loaded_data.shape = ", loaded_data.shape)

# 각 열의 최대값을 찾기 위해 행과 열을 바꾸어 줌. 즉 전치향렬을 만들어줌
transpose_loaded_data = loaded_data.T

print("transpose_loaded_data.shape = ", transpose_loaded_data.shape)

loaded_data.shape =  (470, 18)
transpose_loaded_data.shape =  (18, 470)


## 전치된 행렬에서 각 행의 최대 값을 찾아 나누어 주면서 Normalize 시킨 후에,
## 다시 전치행렬을 만들어서 csv 파일로 저장함

In [4]:
# 전치행렬을 위한 리스트
transpose_normalize_data_list = []

for index in range(len(transpose_loaded_data)):
    
    max_value = np.max(transpose_loaded_data[index, :])   # 각 행의 최대값을 찾음
    
    # 최대값이 1 이상이면 최대값으로 나누어서 
    # 모든 데이터가 0 ~ 1 사이에 오도록 함
    if max_value > 1.0:  
        
        transpose_normalize_data_list.append(transpose_loaded_data[index, :] / max_value)
       
    # 최대값이 1 보다 작으면 해당 값을 그대로 사용함. 
    ## 왜냐하면 1보다 작은 값이면 굳이 바꿀 필요가 없음
    else:      
        
        transpose_normalize_data_list.append(transpose_loaded_data[index, :])
        

# 리스트를 numpy type 으로 변환
transpose_normalize_data = np.array(transpose_normalize_data_list)

print(transpose_normalize_data.shape)

# 데이터 저장을 위해 다시 전치행렬을 통해 행과 열을 바꿈
normalize_data = transpose_normalize_data.T

print(normalize_data.shape)

# normalize 파일저장
np.savetxt('./Normalize_ThoracicSurgery_data.csv', normalize_data, delimiter=',')

(18, 470)
(470, 18)


In [6]:
# 데이터 불러오기

loaded_data = np.loadtxt('./Normalize_ThoracicSurgery_data.csv', delimiter=',', dtype=np.float32)

print("loaded_data.shape = ", loaded_data.shape)

# 임시 저장 리스트
training_data_list = []
test_data_list = []

# 전체 30 % 라는것은 전체 길이에 대해 3 으로 나누어주면 됨
seperation_rate = 3

for index in range(len(loaded_data)):
    
    if (index % seperation_rate) == 0:
        
        test_data_list.append(loaded_data[index])
        
    else:
        
        training_data_list.append(loaded_data[index])
        
# generate training data from training_data_list using np.arrya(...)
training_data = np.array(training_data_list)

# generate test data from test_data_list using np.arrya(...)
test_data = np.array(test_data_list)

# verification shape
print("training_data.shape = ", training_data.shape)
print("test_data.shape = ", test_data.shape)

# save training & test data (.csv)
np.savetxt('./ThoracicSurgery_generated_training_data.csv', training_data, delimiter=',')
np.savetxt('./ThoracicSurgery_generated_test_data.csv', test_data, delimiter=',')

loaded_data.shape =  (470, 18)
training_data.shape =  (313, 18)
test_data.shape =  (157, 18)


In [8]:
#hyper-parameter
i_nodes = training_data.shape[1] - 1    # input nodes 개수
h1_nodes = 30  # hidden nodes 개수. 
o_nodes = 1    # output nodes 개수
lr = 1e-2      # learning rate. hi_node = 30, 
epochs = 30   # 반복횟수. 10번에서는 0.311 에서 미미함

# ThoraricSurgery 객체 생성
obj = ThoracicSurgery(i_nodes, h1_nodes, o_nodes, lr)

print("Neural Network Learning using Numerical Derivative...")

start_time = datetime.now()

for step in range(epochs):
    
    for index in range(len(training_data)):
        
        input_data = training_data[index, 0:-1]
        target_data = training_data[index, [-1]]
        
        obj.train(input_data, target_data)
        
    if (step % 2 == 0):
        print("epochs = ", step, "loss value = ", obj.loss_val())

end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

ThoracicSurgery object is created !!!
Neural Network Learning using Numerical Derivative...
epochs =  0 loss value =  0.26241473395259174
epochs =  2 loss value =  0.26029295205441544
epochs =  4 loss value =  0.25848024437002404
epochs =  6 loss value =  0.2567491125342633
epochs =  8 loss value =  0.25508500780114346
epochs =  10 loss value =  0.2534751791776427
epochs =  12 loss value =  0.2519083245413819
epochs =  14 loss value =  0.2503743079078847
epochs =  16 loss value =  0.24886392818979425
epochs =  18 loss value =  0.2473687282387986
epochs =  20 loss value =  0.2458808355552117
epochs =  22 loss value =  0.2443928280271399
epochs =  24 loss value =  0.2428976195737859
epochs =  26 loss value =  0.24138836174385334
epochs =  28 loss value =  0.2398583582507335

Elapsed Time =>  0:06:13.175000


In [9]:
test_data = np.loadtxt('./ThoracicSurgery_generated_test_data.csv', delimiter=',', dtype=np.float32)
print("test_data.shape = ", test_data.shape)

test_input_data = test_data[ :, 0:-1 ]
test_target_data = test_data[ :, -1 ]

(true_list_1, false_list_1, index_label_prediction_list) = obj.accuracy(test_input_data, test_target_data) 

test_data.shape =  (157, 18)
Accuracy =>  0.8280254777070064


In [10]:
print(index_label_prediction_list)

[[1, 1.0, 0], [10, 1.0, 0], [12, 1.0, 0], [13, 1.0, 0], [15, 1.0, 0], [16, 1.0, 0], [24, 1.0, 0], [32, 1.0, 0], [36, 1.0, 0], [41, 1.0, 0], [49, 1.0, 0], [60, 1.0, 0], [65, 1.0, 0], [66, 1.0, 0], [70, 1.0, 0], [77, 1.0, 0], [80, 1.0, 0], [83, 1.0, 0], [86, 1.0, 0], [97, 1.0, 0], [103, 1.0, 0], [107, 1.0, 0], [115, 1.0, 0], [137, 1.0, 0], [152, 1.0, 0], [154, 1.0, 0], [156, 1.0, 0]]
